# 🌟 YES Ai Studio: Agentic Multimodal Creator (IoT Edition) 🌟
### _An Autonomous AI Agent that Controls the Physical World via Tuya Cloud_

---

### **👨‍💻 Author:** Ranajit Dhar
### **🏆 Event:** Tuya AI Innovators Hackathon 2025 (AI Agent Track)
### **📅 Date:** December 2025

## 🚀 Project Overview
**YES Ai Studio** is a cutting-edge **Agentic AI System** that goes beyond digital content creation. It bridges the gap between **Generative AI (Google Gemini)** and **Smart Home Automation (Tuya IoT Cloud)**.

Unlike traditional "Voice Assistants" that wait for commands, this system is **fully autonomous**. It plans tasks, creates content, **evaluates its own quality**, and decides whether to activate physical studio lighting based on the success of its work.

---

## 🔑 Key Features (The "Brain" & The "Body")

### **1. 🧠 Multi-Agent Orchestration (The Brain)**
- **Navigator Agent:** Plans the workflow and researches trends.
- **Curator Agent:** Generates creative content for the user.
- **Evaluator Agent:** Critically judges the quality (Score 1-10).

### **2. 🔌 Tuya IoT Bridge (The Body)**
- **Autonomous Control:** The Agent connects to **Tuya Open API**.
- **Logic-Based Action:** If `Quality Score >= 8`, the Agent **turns on the Smart Studio Light** (Green Signal).
- **Energy Efficiency:** If quality is low, it keeps the system in "Power Save Mode" (Light OFF).

### **3. 🛠️ Tech Stack**
- **AI Model:** Google Gemini 2.5 Flash
- **IoT Cloud:** TuyaOpen Python SDK (`tuya-connector-python`)
- **Validation:** Pydantic (Structured Outputs)
- **Environment:** Kaggle Notebook (Cloud Native)

### **©️ Copyright 2025 by Ranajit Dhar**
### **📜 License: All Rights Reserved (Proprietary)**

---

In [1]:
# Cell 1: Install Necessary Libraries
!pip install google-genai pydantic pillow pandas matplotlib tuya-connector-python --quiet
print("✅ All Libraries Installed Successfully (AI + IoT Ready).")

✅ All Libraries Installed Successfully (AI + IoT Ready).


In [2]:
# Cell 2: Imports & System Configuration
import os
import json
import time
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

# Google Gemini & Kaggle Imports
from kaggle_secrets import UserSecretsClient
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

# Tuya IoT Imports
from tuya_connector import TuyaOpenAPI

# --- 🔐 CREDENTIALS SETUP ---
print("🔄 Fetching Credentials from Secrets...")
user_secrets = UserSecretsClient()

# 1. Google Gemini Connection
try:
    api_key_value = user_secrets.get_secret("GOOGLE_API_KEY")
    client = genai.Client(api_key=api_key_value)
    print("✅ Google Gemini Client Connected.")
except Exception as e:
    print(f"❌ Error: 'GOOGLE_API_KEY' missing in Secrets. {e}")

# 2. Tuya IoT Connection
try:
    TUYA_ACCESS_ID = user_secrets.get_secret("TUYA_ACCESS_ID")
    TUYA_ACCESS_SECRET = user_secrets.get_secret("TUYA_ACCESS_SECRET")
    TUYA_DEVICE_ID = user_secrets.get_secret("TUYA_DEVICE_ID")
    TUYA_ENDPOINT = "https://openapi.tuyain.com" # India Data Center
    print("✅ Tuya IoT Credentials Fetched.")
except Exception as e:
    print(f"❌ Error: Tuya Secrets are missing! Please check Add-ons -> Secrets.")

🔄 Fetching Credentials from Secrets...
✅ Google Gemini Client Connected.
✅ Tuya IoT Credentials Fetched.


In [3]:
# Cell 3: Memory, Tools, and Pydantic Schemas

# 1. Global Memory (Preserves context)
session_memory = {
    "user_name": "Ranajit Dhar",
    "target_audience": "AI & IoT Developers",
    "recent_goal": "Smart Home Automation with AI Agents",
    "recent_score": None
}

def update_memory(key, value):
    session_memory[key] = value

# 2. Tools (Function Calling)
def get_job_market_trends(career_goal: str) -> str:
    """Fetches market trends for specific career goals."""
    return "Current Trend: High demand for 'Agentic AI' that can control physical IoT hardware."

YES_AI_TOOLS = [get_job_market_trends]

# 3. Schemas (Structured Output)
class Roadmap(BaseModel):
    day: int
    topic: str
    key_tasks: list[str]

class ContentPost(BaseModel):
    title: str
    caption_draft: str
    hashtags: list[str]
    image_prompt: str

class EvaluationScore(BaseModel):
    score_out_of_10: int = Field(description="Score from 1-10 based on quality.")
    critique: str = Field(description="Reasoning for the score.")

print("✅ Core System Architecture (Schemas & Memory) Loaded.")

✅ Core System Architecture (Schemas & Memory) Loaded.


In [4]:
# Cell 4: Define AI Agents

# --- Agent 1: Navigator (Planner) ---
def run_navigator_agent(goal: str):
    print(f"\n--- 🧭 Navigator Agent: Planning for '{goal}' ---")
    prompt = f"Create a 3-day mini roadmap for: {goal}. MUST use `get_job_market_trends` tool."
    
    # Generate with Tool Use
    response = client.models.generate_content(
        model='gemini-2.5-flash', contents=prompt, config=types.GenerateContentConfig(tools=YES_AI_TOOLS)
    )
    
    # Final Structured Output
    final_res = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=f"Create a JSON roadmap for {goal} based on previous context.",
        config=types.GenerateContentConfig(response_mime_type="application/json", response_schema=Roadmap)
    )
    return final_res.text

# --- Agent 2: Curator (Creator) ---
def run_curator_agent(topic: str):
    print(f"\n--- 🎨 Curator Agent: Creating Content ---")
    prompt = f"Create a viral social media post about '{topic}' for developers."
    response = client.models.generate_content(
        model='gemini-2.5-flash', contents=prompt,
        config=types.GenerateContentConfig(response_mime_type="application/json", response_schema=ContentPost)
    )
    update_memory("recent_content", json.loads(response.text))
    return response.text

# --- Agent 3: Evaluator (Judge) ---
def run_evaluator_agent(content: dict):
    print(f"\n--- ⚖️ Evaluator Agent: Quality Check ---")
    prompt = f"Rate this content out of 10: {content['caption_draft']}. Be strict. High score (>8) requires innovation."
    response = client.models.generate_content(
        model='gemini-2.5-flash', contents=prompt,
        config=types.GenerateContentConfig(response_mime_type="application/json", response_schema=EvaluationScore)
    )
    score_data = json.loads(response.text)
    update_memory("recent_score", score_data)
    print(f"   ✅ Quality Score: {score_data['score_out_of_10']}/10")
    return score_data

print("✅ Agents Initialized & Ready.")

✅ Agents Initialized & Ready.


In [5]:
# Cell 5: Execute Multi-Agent Workflow & Visualize Results (The Full Experience)
import time
import pandas as pd
from IPython.display import display, Markdown

print("🚀 Starting YES Ai Studio Workflow...")

# --- 1. RUN NAVIGATOR AGENT (The Planner) ---
try:
    # Goal memory theke nichi
    current_goal = session_memory.get('recent_goal', 'Smart Home Automation')
    nav_output = run_navigator_agent(current_goal)
    print("✅ Navigator Finished Planning.")
except Exception as e:
    print(f"⚠️ Navigator Error: {e}")

# 🛑 PAUSE: Waiting 20s for Google Quota Safety
print("⏳ Processing... (Waiting 20s for API Quota)...")
time.sleep(20) 

# --- 2. RUN CURATOR AGENT (The Creator) ---
try:
    # Navigator er output theke topic nichi
    roadmap_data = session_memory.get('recent_roadmap')
    if roadmap_data and 'recommended_path' in roadmap_data and len(roadmap_data['recommended_path']) > 0:
        topic = roadmap_data['recommended_path'][0]['topic']
    else:
        topic = "AI Agents controlling IoT Devices"
        
    curator_output = run_curator_agent(topic)
    print("✅ Curator Finished Creating Content.")
except Exception as e:
    print(f"⚠️ Curator Error: {e}")

# 🛑 PAUSE: Waiting 20s for Google Quota Safety
print("⏳ Reviewing... (Waiting 20s for API Quota)...")
time.sleep(20)

# --- 3. RUN EVALUATOR AGENT (The Judge) ---
content_data = session_memory.get("recent_content")
if content_data:
    try:
        score_data = run_evaluator_agent(content_data)
        print("✅ Evaluator Finished Scoring.")
    except Exception as e:
        print(f"⚠️ Evaluator Error: {e}")
else:
    print("❌ Error: Content generation failed, skipping evaluation.")


# ==========================================
# 📊 FINAL VISUALIZATION
# ==========================================

print("\n" + "="*40)
print("       🌟 YES AI STUDIO DASHBOARD       ")
print("="*40 + "\n")

# 1. DISPLAY ROADMAP
display(Markdown("## 📋 Final Roadmap from Navigator Agent"))
roadmap_data = session_memory.get('recent_roadmap')

if roadmap_data and 'recommended_path' in roadmap_data and len(roadmap_data['recommended_path']) > 0:
    # Create DataFrame for beautiful table
    df = pd.DataFrame(roadmap_data['recommended_path'])
    display(df[['day', 'topic', 'key_tasks']])
else:
    display(Markdown("⚠️ **Warning:** Navigator Agent could not generate the structured roadmap data. Showing raw output instead."))

# 2. DISPLAY CONTENT & EVALUATION
display(Markdown("## ⭐ Content Post & Evaluation (Curator & Evaluator Agents)"))

content_data = session_memory.get('recent_content', {})
score_data = session_memory.get('recent_score', {})

if 'title' in content_data:
    # Formatting the output nicely with Markdown
    display(Markdown(f"### **Post Title:** {content_data['title']}"))
    display(Markdown(f"**📝 Post Draft:**\n\n_{content_data['caption_draft']}_"))
    
    hashtags = " ".join(content_data.get('hashtags', []))
    display(Markdown(f"**🏷️ Hashtags:** `{hashtags}`"))
    
    display(Markdown("---"))
    
    # Score Display
    final_score = score_data.get('score_out_of_10', 'N/A')
    display(Markdown(f"### **🏆 Quality Score:** <span style='color:blue'>{final_score}/10</span>"))
    
    # Critique Display
    display(Markdown(f"**🕵️ Evaluator Critique:**\n\n{score_data.get('critique', 'No critique available.')}"))

else:
    display(Markdown("❌ **Error:** Content Data missing."))

🚀 Starting YES Ai Studio Workflow...

--- 🧭 Navigator Agent: Planning for 'Smart Home Automation with AI Agents' ---
✅ Navigator Finished Planning.
⏳ Processing... (Waiting 20s for API Quota)...

--- 🎨 Curator Agent: Creating Content ---
✅ Curator Finished Creating Content.
⏳ Reviewing... (Waiting 20s for API Quota)...

--- ⚖️ Evaluator Agent: Quality Check ---
   ✅ Quality Score: 8/10
✅ Evaluator Finished Scoring.

       🌟 YES AI STUDIO DASHBOARD       



## 📋 Final Roadmap from Navigator Agent

⚠️ **Warning:** Navigator Agent could not generate the structured roadmap data. Showing raw output instead.

## ⭐ Content Post & Evaluation (Curator & Evaluator Agents)

### **Post Title:** AI Agents Taking Over Your IoT? Devs, This Changes EVERYTHING!

**📝 Post Draft:**

_Imagine your smart home or industrial IoT setup not just reacting, but *anticipating*. AI Agents are here to autonomously manage, optimize, and learn from your connected devices. From predictive maintenance to hyper-personalized environments, the possibilities are limitless. For developers, this means building more intelligent, self-sufficient systems with less manual oversight. Dive into the future where your code empowers machines to think and act. What's the first autonomous IoT system YOU'd build?_

**🏷️ Hashtags:** `AI IoT DeveloperLife TechInnovation FutureTech SmartDevices Automation Code AIAgents MachineLearning`

---

### **🏆 Quality Score:** <span style='color:blue'>8/10</span>

**🕵️ Evaluator Critique:**

The content is exceptionally well-written, engaging, and highly effective at communicating a compelling vision. It skillfully uses impactful language to describe the innovative potential of AI Agents in IoT, moving beyond mere features to highlight anticipation, autonomy, and hyper-personalization. The benefits are clearly articulated for both end-users and developers, and the call to action is strong. While the *subject matter* is highly innovative, the *content itself* doesn't break new ground in terms of writing style or argumentative structure; it's a very high-quality example of promotional technology content. However, it earns an 8 because it successfully *frames* and *sells* the innovation in a captivating and inspiring way, making the innovative aspect of the technology shine through the writing.

In [6]:
# Cell 6: 🔌 Tuya IoT Bridge - Action Layer (FINAL DEMO VERSION)

print("\n🤖 INITIATING IOT ACTION SEQUENCE...")

# 1. Connect to Tuya Cloud
try:
    openapi = TuyaOpenAPI(TUYA_ENDPOINT, TUYA_ACCESS_ID, TUYA_ACCESS_SECRET)
    openapi.connect()
    print("   ✅ Connected to Tuya Cloud.")
except Exception as e:
    print(f"   ❌ Connection Failed: {e}")

# 2. Retrieve Score Logic
if session_memory.get('recent_score'):
    # Get the actual score from the AI Agent
    final_score = session_memory['recent_score'].get('score_out_of_10', 0) 
    
    print(f"   📊 Analyzed Score: {final_score}")

    # 3. DECISION LOGIC
    if final_score >= 8:
        print("   🚀 DECISION: High Quality -> ACTIVATING STUDIO LIGHTS (Green Signal)")
        
        # Command: Turn Switch ON
        commands = {'commands': [{'code': 'switch', 'value': True}]} 
        
        response = openapi.post(f'/v1.0/iot-03/devices/{TUYA_DEVICE_ID}/commands', commands)
        
        if response.get('success'):
            print("   ✅ SUCCESS: Physical Device Turned ON via Tuya Cloud API.")
            print("   🎉 Great Job! The Studio is now LIVE.")
        else:
            print(f"   ⚠️ API Sent. Response: {response}")

    else:
        print("   📉 DECISION: Quality Low -> DEACTIVATING LIGHTS (Power Save)")
        
        # Command: Turn Switch OFF
        commands = {'commands': [{'code': 'switch', 'value': False}]}
        response = openapi.post(f'/v1.0/iot-03/devices/{TUYA_DEVICE_ID}/commands', commands)
        
        if response.get('success'):
            print("   ✅ SUCCESS: Device Turned OFF.")
        else:
            print(f"   ⚠️ API Sent. Response: {response}")
else:
    print("❌ No score found in memory. Please RUN THE AGENTS (Cell 5) first!")


🤖 INITIATING IOT ACTION SEQUENCE...
   ✅ Connected to Tuya Cloud.
   📊 Analyzed Score: 8
   🚀 DECISION: High Quality -> ACTIVATING STUDIO LIGHTS (Green Signal)
   ✅ SUCCESS: Physical Device Turned ON via Tuya Cloud API.
   🎉 Great Job! The Studio is now LIVE.


## 🏁 Conclusion & Impact
**YES Ai Studio (IoT Edition)** demonstrates the true potential of **Agentic AI** in the physical world.

By integrating **Google Gemini's reasoning capabilities** with the robust **Tuya IoT Ecosystem**, we have moved from simple "Remote Control" to **"Intelligent Automation"**.

### 💡 Why This Matters?
1.  **No Human Intervention:** The AI evaluated the work and controlled the hardware autonomously.
2.  **Scalability:** While demonstrated with a Virtual Switch, this same logic can control industrial machinery, home security, or smart city infrastructure using the Tuya Platform.
3.  **Real-World Logic:** The "Bridge Logic" proves that AI can be trusted to manage physical resources based on quality metrics.

### 🎉 Final Verdict
This project successfully transforms a standard AI workflow into a **Physical Action**, proving that **Tuya + Gemini** is the future of Smart Hardware.

---

In [7]:
# Cell 9: COPYRIGHT AND LICENSE

copyright_text = """
### 📝 COPYRIGHT AND LICENSE

**Copyright (c) 2025 Ranajit Dhar. All Rights Reserved.**

This software and its documentation are the property of Ranajit Dhar. 
No part of this code, software, or documentation may be reproduced, 
distributed, or transmitted in any form or by any means, including 
photocopying, recording, or other electronic or mechanical methods, 
without the prior written permission of the author.

This project is NOT intended for commercial use or redistribution.
"""

with open('LICENSE.txt', 'w', encoding='utf-8') as f:
    f.write(copyright_text.strip())

print("✅ LICENSE.txt File Created")

✅ LICENSE.txt File Created
